# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [1]:
from selenium import webdriver 
from selenium.webdriver.support.ui import Select as s

browser = webdriver.Chrome(executable_path="C:/Users/user/chromedriver.exe")
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

In [4]:
from datetime import timedelta
import datetime 
d1 = datetime.datetime(2018,8,31)
#d2 = datetime.datetime(2018,1,1)
#t =(d1-d2).days
#print(t)
eight_month_ago = d1 - timedelta(days = 242)

In [6]:
sSite = s(browser.find_element_by_id("ctl05_ddlSite"))
sSite.select_by_value("11")
sYear = s(browser.find_element_by_id("ctl05_ddlYear"))
sYear.select_by_value("2018")
browser.find_element_by_id('ctl05_btnQuery').click()

In [20]:
from bs4 import BeautifulSoup as b
url = "https://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx"
html_source = browser.page_source
soup = b(html_source,"html5")
table = soup.find("table",attrs={"class":"TABLE_G"})
so2 = table.find("tbody").find_all("td",attrs={"class":"no-alt"})[0].text

d = {}
for tr in table.find_all("tr")[1:]:
    for i ,td in enumerate(tr.find_all("td")):
        if len(tr.find_all("td")) == 5:
            if i == 0:
                obs = td.text
                d.setdefault(obs,{})
            if i == 2:
                date = td.text #日期
            if i == 3:
                value = td.text
                d[obs][date] = value
        if len(tr.find_all("td")) == 3:
            if i == 0:
                date = td.text #日期
            if i == 1:
                value = td.text
                d[obs][date] = value
                
import pandas as pd
df = pd.DataFrame(d)
df.SO2[0:8]

2018/01    1.80
2018/02    1.90
2018/03    2.20
2018/04    2.30
2018/05    3.10
2018/06    2.70
2018/07    2.20
2018/08    2.40
Name: SO2, dtype: object

### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [21]:
from selenium import webdriver 
from selenium.webdriver.support.ui import Select as s

url = "http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx"
browser = webdriver.Chrome(executable_path="C:/Users/user/chromedriver.exe")
browser.get(url)

In [31]:
sSite = s(browser.find_element_by_id("ctl05_ddlSite"))
sSite.select_by_value("11")
sYear = s(browser.find_element_by_id("ctl05_ddlYear"))
sYear.select_by_value("2018")
browser.find_element_by_id('ctl05_btnQuery').click()

In [52]:
from bs4 import BeautifulSoup
html_source = browser.page_source
soup = b(html_source,"html5")
table = soup.find("table",attrs={"class":"TABLE_G"})

item = {}
for tr in table.find("tbody").find_all("tr")[1:]:
    for i , td in enumerate(tr.find_all("td")):
        if len(tr.find_all("td")) == 5:
            if i == 0:
                obs = td.text
                item.setdefault(obs,{})
            if i == 2:
                date = td.text
            if i == 3:
                value = td.text
                item[obs][date] = value
        if len(tr.find_all("td")) == 3:
            if i == 0:
                date = td.text
            if i == 1:
                value = td.text
                item[obs][date] = value
                
import pandas as pd
df = pd.DataFrame(item)
print(df.SO2[0:8])
print(df.CO[0:8])

2018/01    1.80
2018/02    1.90
2018/03    2.20
2018/04    2.30
2018/05    3.10
2018/06    2.70
2018/07    2.20
2018/08    2.40
Name: SO2, dtype: object
2018/01    0.34
2018/02    0.44
2018/03    0.40
2018/04    0.38
2018/05    0.34
2018/06    0.29
2018/07    0.21
2018/08    0.30
Name: CO, dtype: object
